In [60]:
import torch
from pathlib import Path
import os
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import numpy as np
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import optim
from torch.optim import lr_scheduler
import time
from tqdm.autonotebook import tqdm, trange
from torchvision import models
import gc
import pandas as pd

In [25]:
use_gpu=torch.cuda.is_available()

In [54]:
torch.cuda.empty_cache()
gc.collect()

0

In [53]:
def open_img(path): 
    transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
        ])
    image = Image.open(path)
    image.load()
    image = image.resize((244, 244))
    x = np.array(image)
    x = np.array(x / 255, dtype='float32')
    x = transform(x)
    return x

In [28]:
classes = list(map(lambda x: x.name, Path(r"C:\Users\Stepan\Desktop\симпсоны\train").glob("*")))
classes=sorted(classes)
names={classes[i]:i for i in range(42)}


In [29]:
list_img=[]
for i in classes:
    jpgs=list(map(lambda x: x.name,Path(fr"C:\Users\Stepan\Desktop\симпсоны\train\{i}").glob("*")))
    for j in jpgs:
        list_img.append([open_img(fr"C:\Users\Stepan\Desktop\симпсоны\train\{i}\{j}"), names[i]])

In [30]:
train_set, val_set=train_test_split(list_img, test_size=0.2)
train_loader=DataLoader(train_set, batch_size=4, shuffle=True)
val_loader=DataLoader(val_set, batch_size=4, shuffle=True)
dataloaders={"train":train_loader, "val":val_loader}

In [31]:
model=models.vgg16(pretrained=True)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [32]:
dataset_sizes={x: len(dataloaders[x])*4 for x in ['train','val']}
dataset_sizes

{'train': 16748, 'val': 4188}

In [47]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0
    
    #Ваш код здесь
    losses = {'train': [], "val": []}

    pbar = trange(num_epochs, desc="Epoch:")

    for epoch in pbar:

        # каждя эпоха имеет обучающую и тестовую стадии
        for phase in ['train', 'val']:
            if phase == 'train':
                
                model.train(True)  # установаить модель в режим обучения
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # итерируемся по батчам
            for data in tqdm(dataloaders[phase], leave=False, desc=f"{phase} iter:"):
                # получаем картинки и метки
                inputs, labels = data
                if use_gpu:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                else:
                    inputs, labels = inputs, labels



                # инициализируем градиенты параметров
                if phase=="train":
                    optimizer.zero_grad()

                # forward pass
                if phase == "eval":
                    with torch.no_grad():
                        outputs = model(inputs)
                else:
                    outputs = model(inputs)
                
                preds = torch.argmax(outputs, -1)
                loss = criterion(outputs, labels)

                # backward pass + оптимизируем только если это стадия обучения
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    

                # статистика
                
                running_loss += loss.item()
                running_corrects += int(torch.sum(preds == labels.data))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]
            print(epoch_loss, epoch_acc)
            # Ваш код здесь
            losses[phase].append(epoch_loss)
            print(running_corrects, dataset_sizes[phase], phase)
            pbar.set_description('{} Loss: {:.4f} Acc: {:.4f}'.format(
                                    phase, epoch_loss, epoch_acc
                                ))

            # если достиглось лучшее качество, то запомним веса модели
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
        scheduler.step()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # загрузим лучшие веса модели
    model.load_state_dict(best_model_wts)
    return model, losses

In [48]:
for param in model.features:
    param.reqiures_grad=False

In [49]:
model.classifier= nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(25088, 2048),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(1024, 42))
model=model.cuda()

In [50]:

# В качестве cost function используем кросс-энтропию
loss_fn = nn.CrossEntropyLoss()

# В качестве оптимизатора - стохастический градиентный спуск
optimizer_ft = optim.Adam(model.parameters(), lr=1e-4)

# Умножает learning_rate на 0.1 каждые 7 эпох (это одна из эвристик, не было на лекциях)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [55]:
model, losses = train_model(model, loss_fn, optimizer_ft, exp_lr_scheduler, num_epochs=16)

Epoch::   0%|          | 0/16 [00:00<?, ?it/s]

train iter::   0%|          | 0/4187 [00:00<?, ?it/s]

0.1288708177428705 0.8885240028660139
14881 16748 train


val iter::   0%|          | 0/1047 [00:00<?, ?it/s]

0.11717244992631444 0.8999522445081184
3769 4188 val


...

0.1505646425534542 0.9565425023877746
4006 4188 val


train iter::   0%|          | 0/4187 [00:00<?, ?it/s]

0.0008225679716455128 0.9989849534272749
16731 16748 train


val iter::   0%|          | 0/1047 [00:00<?, ?it/s]

0.1504212300181006 0.9567812798471824
4007 4188 val
Training complete in 287m 8s
Best val Acc: 0.956781


In [134]:
test_img=[]

jpgs_test=list(map(lambda x: x.name,Path(r"C:\Users\Stepan\Desktop\симпсоны\testset\Новая папка").glob("*")))
jpgs_test=sorted(jpgs_test)
for i in range(991):
    test_img.append(open_img(fr"C:\Users\Stepan\Desktop\симпсоны\testset\Новая папка\img{i}.jpg"))
data_test=DataLoader(test_img, batch_size=4, shuffle=False)

In [133]:
sorted(jpgs_test)

['img0.jpg',
 'img1.jpg',
 'img10.jpg',
 'img100.jpg',
 'img101.jpg',
 'img102.jpg',
 'img103.jpg',
 'img104.jpg',
 'img105.jpg',
 'img106.jpg',
 'img107.jpg',
 'img108.jpg',
 'img109.jpg',
 'img11.jpg',
 'img110.jpg',
 'img111.jpg',
 'img112.jpg',
 'img113.jpg',
 'img114.jpg',
 'img115.jpg',
 'img116.jpg',
 'img117.jpg',
 'img118.jpg',
 'img119.jpg',
 'img12.jpg',
 'img120.jpg',
 'img121.jpg',
 'img122.jpg',
 'img123.jpg',
 'img124.jpg',
 'img125.jpg',
 'img126.jpg',
 'img127.jpg',
 'img128.jpg',
 'img129.jpg',
 'img13.jpg',
 'img130.jpg',
 'img131.jpg',
 'img132.jpg',
 'img133.jpg',
 'img134.jpg',
 'img135.jpg',
 'img136.jpg',
 'img137.jpg',
 'img138.jpg',
 'img139.jpg',
 'img14.jpg',
 'img140.jpg',
 'img141.jpg',
 'img142.jpg',
 'img143.jpg',
 'img144.jpg',
 'img145.jpg',
 'img146.jpg',
 'img147.jpg',
 'img148.jpg',
 'img149.jpg',
 'img15.jpg',
 'img150.jpg',
 'img151.jpg',
 'img152.jpg',
 'img153.jpg',
 'img154.jpg',
 'img155.jpg',
 'img156.jpg',
 'img157.jpg',
 'img158.jpg',
 'img

In [61]:
sub=pd.read_csv(r"C:\Users\Stepan\Desktop\симпсоны\sample_submission.csv")
sub

,Id,Expected
0,img0.jpg,bart_simpson
1,img1.jpg,bart_simpson
2,img2.jpg,bart_simpson
3,img3.jpg,bart_simpson
4,img4.jpg,bart_simpson
...,...,...
986,img986.jpg,bart_simpson
987,img987.jpg,bart_simpson
988,img988.jpg,bart_simpson
989,img989.jpg,bart_simpson


In [135]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []
        for inputs in test_loader:
            model.eval()
            outputs = model(inputs.cuda())
            logits.append(outputs.to('cpu'))
            
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs
probs = predict(model, data_test)
preds = np.argmax(probs, axis=1)
    


In [136]:
preds

array([29,  4, 24, 29, 20, 32, 17,  6, 32,  9, 28, 28, 15,  0, 29,  6, 24,
        9, 15, 25, 22, 32, 27,  7,  7,  2,  0, 11, 15, 27, 17,  9,  6, 22,
       11, 37,  7, 27, 27, 27, 27, 29,  9, 24, 11, 11, 17, 32, 24, 25, 20,
        9, 25, 17, 25, 37, 29, 20,  6,  4,  4,  7,  7,  0, 11,  0, 32, 25,
       24,  4,  0, 20, 16, 16,  0, 17, 28, 11,  6, 24, 25, 32,  4,  4,  7,
       22, 20, 20,  7, 37, 29, 28, 32,  4, 20,  7, 32, 17,  6,  7,  7,  2,
       16, 11,  7, 20, 16, 18, 22, 16, 27,  9,  0, 11, 16, 20, 22, 29,  9,
       24,  2, 17, 17, 28, 11,  2,  7,  2, 28, 22, 15, 37, 20,  6,  7, 16,
        0, 25, 18,  0,  9,  7, 29, 24,  6,  2, 24,  4, 11, 11, 28,  7, 15,
        6, 22,  2, 18, 18,  9, 16, 37, 28, 28,  9,  6, 11,  4, 16, 28, 17,
        7, 24, 28, 17, 37, 28,  7, 29,  0, 17, 17, 20, 17, 24, 20, 27, 29,
       22,  4,  7,  7, 16, 22, 20,  2,  6, 22,  2, 22,  2, 11, 15, 15,  4,
       32, 11, 25, 27, 18, 27, 15, 18, 20, 37,  7,  2, 24,  6, 24, 16, 15,
       20, 25,  2, 24,  6

In [139]:
d=dict()
for i in range(42):
    d[i]=classes[i]
itog=list(map(lambda x: d[x], preds))

In [140]:
itog

['nelson_muntz',
 'bart_simpson',
 'mayor_quimby',
 'nelson_muntz',
 'lisa_simpson',
 'principal_skinner',
 'krusty_the_clown',
 'charles_montgomery_burns',
 'principal_skinner',
 'comic_book_guy',
 'ned_flanders',
 'ned_flanders',
 'homer_simpson',
 'abraham_grampa_simpson',
 'nelson_muntz',
 'charles_montgomery_burns',
 'mayor_quimby',
 'comic_book_guy',
 'homer_simpson',
 'milhouse_van_houten',
 'marge_simpson',
 'principal_skinner',
 'moe_szyslak',
 'chief_wiggum',
 'chief_wiggum',
 'apu_nahasapeemapetilon',
 'abraham_grampa_simpson',
 'edna_krabappel',
 'homer_simpson',
 'moe_szyslak',
 'krusty_the_clown',
 'comic_book_guy',
 'charles_montgomery_burns',
 'marge_simpson',
 'edna_krabappel',
 'sideshow_bob',
 'chief_wiggum',
 'moe_szyslak',
 'moe_szyslak',
 'moe_szyslak',
 'moe_szyslak',
 'nelson_muntz',
 'comic_book_guy',
 'mayor_quimby',
 'edna_krabappel',
 'edna_krabappel',
 'krusty_the_clown',
 'principal_skinner',
 'mayor_quimby',
 'milhouse_van_houten',
 'lisa_simpson',
 'comi

In [141]:
sub['Expected']=itog

In [143]:
sub.index=sub['Id']
sub3=sub.drop('Id', axis=1)
sub3.to_csv(r"C:\Users\Stepan\Desktop\симпсоны\5.csv")

In [144]:
sub3

,Expected
Id,
img0.jpg,nelson_muntz
img1.jpg,bart_simpson
img2.jpg,mayor_quimby
img3.jpg,nelson_muntz
img4.jpg,lisa_simpson
...,...
img986.jpg,sideshow_bob
img987.jpg,nelson_muntz
img988.jpg,ned_flanders
